In [2]:
import requests
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster

# Gathering Lat/Long Coordinates with Mapquest's API:

In [3]:
sample_data = ['woodstream blvd & sweetwater blvd, Sugar Land, TX, US', 'Richmond Ave & Weslayan St, Houston, TX, US', 'Buffalo Speedway & N Braeswood Blvd, Houston, TX, US']
sample_df = pd.DataFrame(sample_data, columns = ['locations'])

In [4]:
sample_df

,locations
0,"woodstream blvd & sweetwater blvd, Sugar Land,..."
1,"Richmond Ave & Weslayan St, Houston, TX, US"
2,"Buffalo Speedway & N Braeswood Blvd, Houston, ..."


In [5]:
# Mapquest API key:
key = 'rzzXr0hAl9W1GfRirlBfO1Wm4ihkXf5u'

modified_locations = []

latitude_list = []
longitude_list = []

def location_cleaning(df_column):
    for string_item in df_column:
        item_modified = string_item.replace(' ', '+').replace('&', '%26').replace(',', '%2C')
        modified_locations.append(item_modified)
    for item in modified_locations:
        base_url = f'https://www.mapquestapi.com/geocoding/v1/address?key={key}&inFormat=kvp&outFormat=json&location={item}&thumbMaps=false'
        output = requests.get(base_url).json()
        latitude_list.append(output['results'][0]['locations'][0]['latLng']['lat'])
        longitude_list.append(output['results'][0]['locations'][0]['latLng']['lng'])
    sample_df['latitude'] = pd.DataFrame(latitude_list)
    sample_df['longitude'] = pd.DataFrame(longitude_list)

In [6]:
# should look like this:
# 'Richmond+Ave+%26+Weslayan+St%2C+Houston%2C+TX%2C+US'

modified_locations

[]

In [7]:
location_cleaning(sample_df['locations'])

In [8]:
sample_df

,locations,latitude,longitude
0,"woodstream blvd & sweetwater blvd, Sugar Land,...",29.576906,-95.617610
1,"Richmond Ave & Weslayan St, Houston, TX, US",29.732737,-95.441494
2,"Buffalo Speedway & N Braeswood Blvd, Houston, ...",29.696450,-95.427222


# Mapping with Folium

In [9]:
map = folium.Map(
    location=[29.732737, -95.441494], # these coordinates are for VIEWING ONLY! Plot coordinates below.
    tiles='OpenStreetMap', # try 'CartoDB dark_matter' or 'cartodbpositron'
    # check out others here: https://python-graph-gallery.com/288-map-background-with-folium/
    zoom_start=10,
)
sample_df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map), axis=1)
# from Abdishakur
FastMarkerCluster(list(zip(sample_df['latitude'].values, sample_df['longitude'].values))).add_to(map)
folium.LayerControl().add_to(map)
map